# Habrahabr news scrapper

Magic функция, которая будет одновременно выполняет код ячейки и экспортирует его в файл, так как встроенная функция %%writefile такой функциональности не поддерживает.

In [1]:
from IPython.core.magic import register_cell_magic


@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = "w"
    if len(argz) == 2 and argz[0] == "-a":
        mode = "a"
    with open(file, mode) as f:
        f.write(cell + "\n\n")
    get_ipython().run_cell(cell)

Секция импорта

In [2]:
%%write_and_run scrapper.py
import locale
import time
import unicodedata

import requests
from fake_useragent import UserAgent
from lxml import etree, html
from pymongo import MongoClient

locale.setlocale(locale.LC_TIME, "ru_RU.UTF-8")

'ru_RU.UTF-8'

Скачиваем и парсим данные

In [3]:
%%write_and_run -a scrapper.py
# fake agent
user_agent = UserAgent().chrome

In [4]:
%%write_and_run -a scrapper.py
# init requests session
session = requests.Session()
session.headers.update(
    {"User-Agent": user_agent,}
)

In [5]:
%%write_and_run -a scrapper.py
# download page
url = "https://habr.com/ru/news/"
page = session.get(url)
content = page.text
parsed = html.fromstring(content)

In [6]:
%%write_and_run -a scrapper.py
# parse page
data = []
headers = parsed.xpath('//h2[@class="post__title"]')
for header in headers:
    _id = header.xpath('./a[@class="post__title_link"]/@href')[0]
    text = header.xpath('./a[@class="post__title_link"]/text()')[0]
    text = unicodedata.normalize("NFKD", text)
    data.append({"_id": _id, "text": text})

Сохраняем данные в БД

In [7]:
%%write_and_run -a scrapper.py
# make connection to mongo
mongo = MongoClient("localhost", 27017)
db = mongo.habr
collection = db.news

In [8]:
%%write_and_run -a scrapper.py
# update database
for item in data:
    if collection.update_one({'_id': item['_id']}, {"$set": item}, upsert=True).matched_count > 0:
        break

Полезные операции с монгой, чтобы каждый раз не набирать в консоли

In [ ]:
# Создание контейнера
!docker run -d -p 27017:27017 --name mongodb mongo 

In [ ]:
# Запуск контейнера
!docker start mongodb

In [ ]:
# Дамп базы 'habr' в рабочую директорию
!docker exec -it mongodb mongodump --out=/backup/ --db=habr
!docker exec -it mongodb tar czf dump.mongo.tgz /backup
!docker cp mongodb:/dump.mongo.tgz dump.mongo.tgz
!docker exec -it mongodb rm -rf /backup /dump.mongo.tgz

In [ ]:
# Восстановление данных из дампа
!docker cp dump.mongo.tgz mongodb:/dump.mongo.tgz
!docker exec -it mongodb tar xzf dump.mongo.tgz
!docker exec -it mongodb mongorestore /backup
!docker exec -it mongodb rm -rf /backup /dump.mongo.tgz

In [ ]:
# Остановка контейнера
!docker stop mongodb